# Реализуем методы для наивного байеса

Сгенерируем выборку, в которой каждый признак имеет некоторое своё распределение, параметры которого отличаются для каждого класса. Затем реализуем несколько методов для класса, который уже частично написан ниже:
- метод predict
- метод \_find\_expon\_params и \_get\_expon\_density для экспоненциального распределения
- метод \_find\_norm\_params и \_get\_norm\_probability для биномиального распределения

Для имплементации \_find\_something\_params изучите документацию функций для работы с этими распределениями в scipy.stats и используйте предоставленные там методы.

In [1]:
import numpy as np
import scipy
import scipy.stats

Сформируем параметры генерации для трех датасетов

In [2]:
func_params_set0 = [(scipy.stats.bernoulli, [dict(p=0.1), dict(p=0.5)]),
                   ]

func_params_set1 = [(scipy.stats.bernoulli, [dict(p=0.1), dict(p=0.5)]),
                    (scipy.stats.expon, [dict(scale=1), dict(scale=0.3)]),
                   ]

func_params_set2 = [(scipy.stats.bernoulli, [dict(p=0.1), dict(p=0.5)]),
                    (scipy.stats.expon, [dict(scale=1), dict(scale=0.3)]),
                    (scipy.stats.norm, [dict(loc=0, scale=1), dict(loc=1, scale=2)]),
                   ]

def generate_dataset_for_nb(func_params_set=[], size = 2500, random_seed=0):
    '''
    Генерирует выборку с заданными параметрами распределений P(x|y).
    Число классов задается длиной списка с параметрами.
    Возвращает X, y, список с названиями распределений
    '''
    np.random.seed(random_seed)

    X = []
    names = []
    for func, params in func_params_set:
        names.append(func.name)
        f = []
        for i, param in enumerate(params):
            f.append(func.rvs(size=size, **param))
        f = np.concatenate(f).reshape(-1,1)
        X.append(f)

    X = np.concatenate(X, 1)
    y = np.array([0] * size + [1] * size)

    shuffle_inds = np.random.choice(range(len(X)), size=len(X), replace=False)
    X = X[shuffle_inds]
    y = y[shuffle_inds]

    return X, y, names 

X, y, distrubution_names = generate_dataset_for_nb(func_params_set0)
X.shape, y.shape, distrubution_names

((5000, 1), (5000,), ['bernoulli'])

In [3]:
from collections import defaultdict
from sklearn.base import BaseEstimator, ClassifierMixin

class NaiveBayes(BaseEstimator, ClassifierMixin):
    '''
    Реализация наивного байеса, которая помимо X, y
    принимает на вход во время обучения 
    виды распределений значений признаков
    '''
    def __init__(self):
        pass
    
    def _find_bernoulli_params(self, x):
        '''
        метод возвращает найденный параметр `p`
        распределения scipy.stats.bernoulli
        '''
        return dict(p=np.mean(x))
    
    def _get_bernoulli_probability(self, x, params):
        '''
        метод возвращает вероятность x для данных
        параметров распределния
        '''
        return scipy.stats.bernoulli.pmf(x, **params)

    def _find_expon_params(self, x):
        # нужно определить параметры распределения
        # и вернуть их
        pass
    
    def _get_expon_density(self, x, params):
        # нужно вернуть плотность распределения в x
        pass

    def _find_norm_params(self, x):
        # нужно определить параметры распределения
        # и вернуть их
        pass
    
    def _get_norm_density(self, x, params):
        # нужно вернуть плотность распределения в x
        pass

    def _get_params(self, x, distribution):
        '''
        x - значения из распределения,
        distribution - название распределения в scipy.stats
        '''
        if distribution == 'bernoulli':
            return self._find_bernoulli_params(x)
        elif distribution == 'expon':
            return self._find_expon_params(x)
        elif distribution == 'norm':
            return self._find_norm_params(x)
        else:
            raise NotImplementedError('Unknown distribution')
            
    def _get_probability_or_density(self, x, distribution, params):
        '''
        x - значения,
        distribytion - название распределения в scipy.stats,
        params - параметры распределения
        '''
        if distribution == 'bernoulli':
            return self._get_bernoulli_probability(x, params)
        elif distribution == 'expon':
            return self._get_expon_density(x, params)
        elif distribution == 'norm':
            return self._get_norm_density(x, params)
        else:
            raise NotImplementedError('Unknown distribution')

    def fit(self, X, y, distrubution_names):
        '''
        X - обучающая выборка,
        y - целевая переменная,
        feature_distributions - список названий распределений, 
        по которым предположительно распределны значения P(x|y)
        ''' 
        assert X.shape[1] == len(distrubution_names)
        assert set(y) == {0, 1}
        self.n_classes = len(np.unique(y))
        self.distrubution_names = distrubution_names
        
        self.y_prior = [(y == j).mean() for j in range(self.n_classes)]
        
        self.distributions_params = defaultdict(dict)
        for i in range(X.shape[1]):
            distribution = self.distrubution_names[i]
            for j in range(self.n_classes):
                values = X[y == j, i]
                self.distributions_params[j][i] = \
                    self._get_params(values, distribution)
        
        return self.distributions_params
    
    def predict(self, X):
        '''
        X - тестовая выборка
        '''
        assert X.shape[1] == len(self.distrubution_names)
        
        # нужно реализовать подсчет аргмаксной формулы, по которой 
        # наивный байес принимает решение о принадлежности объекта классу
        # и применить её для каждого объекта в X
        #
        # примечание: обычно подсчет этой формулы реализуют через 
        # её логарифмирование, то есть, через сумму логарифмов вероятностей, 
        # поскольку перемножение достаточно малых вероятностей будет вести
        # к вычислительным неточностям
        
        return preds

Проверим результат на примере первого распределения

In [4]:
nb = NaiveBayes()
nb.fit(X, y, ['bernoulli'])

defaultdict(dict, {0: {0: {'p': 0.1128}}, 1: {0: {'p': 0.482}}})

In [5]:
from sklearn.metrics import f1_score

prediction = nb.predict(X)
score = f1_score(y, prediction)
print('{:.4f}'.format(score))

0.6045


# Ответы для формы

Ответом для формы должны служить числа, которые будут выведены ниже. Все ответы проверены: в этих примерах получается одинаковый результат и через сумму логарифмов, и через произведение вероятностей.

In [ ]:
scipy.stats.bernoulli.name

for fps in (func_params_set0 * 2,
            func_params_set1, 
            func_params_set2):
    

    X, y, distrubution_names = generate_dataset_for_nb(fps)
    
    nb = NaiveBayes()
    nb.fit(X, y, distrubution_names)
    prediction = nb.predict(X)
    score = f1_score(y, prediction)
    print('{:.4f}'.format(score))